In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!kaggle competitions download -c santander-customer-transaction-prediction

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /content/gdrive/My Drive/Kaggle. Or use the environment method.


In [ ]:
!unzip 'test.csv.zip'
!unzip 'train.csv.zip'

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata


# TAKE FROM YAG320'S KERNEL
# https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split


# only testing data has fake data
df_test = pd.read_csv('test.csv')
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print('Found',len(real_samples_indexes),'real test')
print('Found',len(synthetic_samples_indexes),'fake test')


d = {}
for i in range(200): d['var_'+str(i)] = 'float32'
d['target'] = 'uint8'
d['ID_code'] = 'object'

train = pd.read_csv('train.csv', dtype=d)
test = pd.read_csv('test.csv', dtype=d)

print('Loaded',len(train),'rows of train')
print('Loaded',len(test),'rows of test')

Found 100000 real test
Found 100000 fake test
Loaded 200000 rows of train
Loaded 200000 rows of test


In [7]:
# simple variance transform
train_2 = pd.read_csv('train.csv')
test_2 = pd.read_csv('test.csv')

ids = np.arange(test_2.shape[0])
real_ids = list(set(ids) - set(synthetic_samples_indexes))
real_test = test_2.iloc[real_ids]
fake_test = test_2.iloc[synthetic_samples_indexes]
real_test_id = real_test.ID_code
features = [c for c in train_2.columns if c not in ['target', 'ID_code']]
df = pd.concat([train_2,real_test], axis = 0)

# FE
for feat in features:
    df[feat+'_var'] = df.groupby([feat])[feat].transform('var')
    df[feat+'plus_'] = df[feat] + df[feat+'_var']
    #df[feat+"minus"] = df[feat] - df[feat+'_var']
    
drop_features = [c for c in df.columns if '_var' in c]
df.drop(drop_features, axis=1, inplace=True)



In [8]:
train = df.iloc[:train_2.shape[0]]
real_test = df.iloc[train_2.shape[0]:]

features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target'] 

print(train.shape)
print(real_test.shape)
print(len(features))

(200000, 402)
(100000, 402)
400


In [ ]:
train['target'].value_counts()

0.0    179902
1.0     20098
Name: target, dtype: int64

In [9]:
bayesian_tr_index, bayesian_val_index  = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(train, train['target'].values))[0]

In [10]:
# shuffle func
def disarrange(a, axis=-1):
    """
    Shuffle `a` in-place along the given axis.
    Apply numpy.random.shuffle to the given axis of `a`.
    Each one-dimensional slice is shuffled independently.
    """
    b = a.swapaxes(axis, -1)
    # Shuffle `b` in-place along the last axis.  `b` is a view of `a`,
    # so `a` is shuffled in place, too.
    shp = b.shape[:-1]
    for ndx in np.ndindex(shp):
        np.random.shuffle(b[ndx])
    return

def augment_fast2(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        disarrange(x1,axis=0)
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        disarrange(x1,axis=0)
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [11]:
def LGB_bayesian(
    num_leaves,  # int
    min_data_in_leaf,  # int
    learning_rate,
    min_sum_hessian_in_leaf,    # int  
    feature_fraction,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
    max_depth):
    
    # LightGBM expects next three parameters need to be integer. So we make them integer
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int

    param = {
        'num_leaves': num_leaves,
        'max_bin': 63,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': learning_rate,
        'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
        'bagging_fraction': 1.0,
        'bagging_freq': 5,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'max_depth': max_depth,
        'save_binary': True, 
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,   

    }    
    
    
    xg_train = lgb.Dataset(train.iloc[bayesian_tr_index][features].values,
                           label=train.iloc[bayesian_tr_index]['target'].values,
                           feature_name=features,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(train.iloc[bayesian_val_index][features].values,
                           label=train.iloc[bayesian_val_index]['target'].values,
                           feature_name=features,
                           free_raw_data = False
                           )   

    num_round = 5000
    clf = lgb.train(param, xg_train, num_round, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    
    predictions = clf.predict(train.iloc[bayesian_val_index][features].values, num_iteration=clf.best_iteration)   
    
    score = metrics.roc_auc_score(train.iloc[bayesian_val_index]['target'].values, predictions)
    
    return score

In [12]:
bounds_LGB = {
    'num_leaves': (5, 20), 
    'min_data_in_leaf': (50, 100),  
    'learning_rate': (0.01, 0.3),
    'min_sum_hessian_in_leaf': (0.00001, 0.01),    
    'feature_fraction': (0.05, 0.5),
    'lambda_l1': (0, 5.0), 
    'lambda_l2': (0, 5.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(-2,15),
}

In [ ]:
!pip install bayesian-optimization
!pip install scikit-optimize

In [16]:
from bayes_opt import BayesianOptimization
import warnings
from sklearn import metrics
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)

init_points = 5
n_iter = 5
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[185]	valid_0's auc: 0.901423
|  1        |  0.9014   |  0.4      |  1.188    |  4.121    |  0.2901   |  14.53    |  72.67    |  0.609    |  0.007758 |  14.62    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.857904
[500]	valid_0's auc: 0.884101
[750]	valid_0's auc: 0.895732
[1000]	valid_0's auc: 0.902479
[1250]	valid_0's auc: 0.906664
[1500]	valid_0's auc: 0.909439
[1750]	valid_0's auc: 0.911096
[2000]	valid_0's auc: 0.911983
[2250]	valid_0's auc: 0.912426
[2

In [18]:
param_lgb = {
        'num_leaves': int(LGB_BO.max['params']['num_leaves']), # remember to int here
        'max_bin': 63,
        'min_data_in_leaf': int(LGB_BO.max['params']['min_data_in_leaf']), # remember to int here
        'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_sum_hessian_in_leaf': LGB_BO.max['params']['min_sum_hessian_in_leaf'],
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': LGB_BO.max['params']['feature_fraction'],
        'lambda_l1': LGB_BO.max['params']['lambda_l1'],
        'lambda_l2': LGB_BO.max['params']['lambda_l2'],
        'min_gain_to_split': LGB_BO.max['params']['min_gain_to_split'],
        'max_depth': int(LGB_BO.max['params']['max_depth']), # remember to int here
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [19]:
param_lgb

{'bagging_fraction': 1.0,
 'bagging_freq': 5,
 'bagging_seed': 1337,
 'boost_from_average': False,
 'boosting_type': 'gbdt',
 'data_random_seed': 1337,
 'drop_seed': 1337,
 'feature_fraction': 0.2730194592821652,
 'feature_fraction_seed': 1337,
 'is_unbalance': True,
 'lambda_l1': 4.9628108570303056,
 'lambda_l2': 4.973461314202421,
 'learning_rate': 0.10294164271466658,
 'max_bin': 63,
 'max_depth': 2,
 'metric': 'auc',
 'min_data_in_leaf': 50,
 'min_gain_to_split': 0.017368427739412384,
 'min_sum_hessian_in_leaf': 0.0004595933246590828,
 'num_leaves': 5,
 'objective': 'binary',
 'save_binary': True,
 'seed': 1337,
 'verbose': 1}

In [21]:
# LGB with CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = train[['ID_code', 'target']]
oof['predict'] = 0
predictions = real_test[['ID_code']]
val_aucs = []
feature_importance_df = pd.DataFrame()
features = [col for col in train.columns if col not in ['target', 'ID_code']]
X_test = real_test[features].values
for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train['target'])):
    X_train, y_train = train.iloc[trn_idx][features], train.iloc[trn_idx]['target']
    X_valid, y_valid = train.iloc[val_idx][features], train.iloc[val_idx]['target']
    
    N = 5
    p_valid,yp = 0,0
    for i in range(N):
        X_t, y_t = augment_fast2(X_train.values, y_train.values)
        X_t = pd.DataFrame(X_t)
        X_t = X_t.add_prefix('var_')
    
        trn_data = lgb.Dataset(X_t, label=y_t)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        lgb_clf = lgb.train(param_lgb,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=3000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = lgb_clf.feature_importance()
    fold_importance_df["fold"] = fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    oof['predict'][val_idx] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold+1)] = yp/N

Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.94934	valid_1's auc: 0.907805
[2000]	training's auc: 0.960309	valid_1's auc: 0.917708
[3000]	training's auc: 0.964828	valid_1's auc: 0.920255
[4000]	training's auc: 0.967718	valid_1's auc: 0.921028
[5000]	training's auc: 0.970002	valid_1's auc: 0.921131
[6000]	training's auc: 0.972014	valid_1's auc: 0.921084
[7000]	training's auc: 0.973916	valid_1's auc: 0.920783
Early stopping, best iteration is:
[4843]	training's auc: 0.969673	valid_1's auc: 0.9212
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.949298	valid_1's auc: 0.90814
[2000]	training's auc: 0.960106	valid_1's auc: 0.917409
[3000]	training's auc: 0.964665	valid_1's auc: 0.919736
[4000]	training's auc: 0.96754	valid_1's auc: 0.920322
[5000]	training's auc: 0.969847	valid_1's auc: 0.920418
[6000]	training's auc: 0.971908	valid_1's auc: 0.920196
[7000]	training's auc: 0.973814	valid_1's auc: 0.92001
[80

In [22]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)

sub = pd.DataFrame()
sub['ID_code']=real_test['ID_code']
sub['target']=predictions['target']
submission = pd.read_csv( 'sample_submission.csv')
new_submission = pd.concat([submission.drop(sub.index,axis = 0),sub],axis=0).sort_index()
new_submission.to_csv('submission_6.csv', index=False)